In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

import warnings
warnings.filterwarnings('ignore')

# Display and plotting defaults
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")



: 

In [2]:
# Load dataset from Hugging Face - direct CSV download
import requests
import io

url = "https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset/resolve/main/dataset.csv"
print("Downloading dataset from Hugging Face...")
response = requests.get(url, timeout=30)
response.raise_for_status()
df = pd.read_csv(io.StringIO(response.text))
print("Dataset loaded successfully!")

# Basic cleaning: drop obvious duplicates and reset index
df = df.drop_duplicates().reset_index(drop=True)

print(f"\nDataset shape: {df.shape}")
print(f"Number of columns: {len(df.columns)}")
df.head()


Dataset loaded successfully!

Dataset shape: (114000, 21)
Number of columns: 21


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
# Prepare data for neural network
# Select audio features as predictors (exclude danceability since it's the target)
num_features = [
    'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'
]

available = [c for c in num_features if c in df.columns]

# Keep rows with no NaNs in used columns and danceability (target)
model_df = df.dropna(subset=available + ['danceability']).copy()

# Inputs: audio features; Target: danceability continuous
X = model_df[available].values
y_continuous = model_df['danceability'].values

print(f"Features used (inputs): {available}")
print(f"X shape: {X.shape}")
print(f"Danceability stats (full data): min={y_continuous.min():.3f}, max={y_continuous.max():.3f}, median={np.median(y_continuous):.3f}")


Features used: ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
Data shape: (114000, 10)

Number of unique genres: 114

Genre distribution (top 20):
track_genre
acoustic             1000
punk-rock            1000
progressive-house    1000
power-pop            1000
pop                  1000
pop-film             1000
piano                1000
party                1000
pagode               1000
opera                1000
new-age              1000
mpb                  1000
minimal-techno       1000
metalcore            1000
metal                1000
mandopop             1000
malay                1000
latino               1000
latin                1000
kids                 1000
Name: count, dtype: int64

All genres: ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill',

In [ ]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train_cont, y_temp_cont = train_test_split(
    X, y_continuous, test_size=0.3, random_state=42
)
X_val, X_test, y_val_cont, y_test_cont = train_test_split(
    X_temp, y_temp_cont, test_size=0.5, random_state=42
)

# Compute median on training set only (to avoid leakage)
median_dance = np.median(y_train_cont)
print(f"Training-set median danceability: {median_dance:.3f}")

# Create binary labels: 1 if >= median, 0 otherwise
y_train = (y_train_cont >= median_dance).astype(int)
y_val = (y_val_cont >= median_dance).astype(int)
y_test = (y_test_cont >= median_dance).astype(int)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert to categorical (one-hot encoding) for neural network
num_classes = 2
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_val_cat = keras.utils.to_categorical(y_val, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

print(f"Training set: {X_train_scaled.shape[0]} samples")
print(f"Validation set: {X_val_scaled.shape[0]} samples")
print(f"Test set: {X_test_scaled.shape[0]} samples")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Number of classes: {num_classes}")


NameError: name 'keras' is not defined

## Part 1: Building the Neural Network Architecture


In [ ]:
# Function to create neural network model
def create_model(learning_rate=0.001, hidden_units=64, dropout_rate=0.3):
    """
    Create a feedforward neural network for binary classification (danceability >= median vs < median).
    
    Parameters:
    - learning_rate: Learning rate for optimizer
    - hidden_units: Number of units in hidden layers
    - dropout_rate: Dropout rate for regularization
    """
    model = models.Sequential([
        layers.Dense(hidden_units, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(dropout_rate),
        layers.Dense(hidden_units, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(hidden_units // 2, activation='relu'),
        layers.Dropout(dropout_rate / 2),
        layers.Dense(num_classes, activation='softmax')  # num_classes=2
    ])
    
    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create initial model to show architecture
initial_model = create_model(learning_rate=0.001, hidden_units=64, dropout_rate=0.3)
print("Neural Network Architecture:")
initial_model.summary()


## Part 2: Hyperparameter Tuning - Multi-Parameter Search (Fast)


In [ ]:
# Hyperparameter tuning: multi-parameter search (FAST)
# Goal: stay fast by using only 10,000 training samples + early stopping + limited trials

import itertools
import gc

# (Optional) reproducibility
try:
    tf.keras.utils.set_random_seed(42)
except Exception:
    pass

sample_size = min(10000, len(X_train_scaled))
if sample_size < len(X_train_scaled):
    print(f"Using sample of {sample_size} training samples for faster hyperparameter search...")
    np.random.seed(42)
    sample_indices = np.random.choice(len(X_train_scaled), size=sample_size, replace=False)
    X_train_sample = X_train_scaled[sample_indices]
    y_train_sample = y_train_cat[sample_indices]
else:
    X_train_sample = X_train_scaled
    y_train_sample = y_train_cat

# Define a small grid (we'll optionally subsample trials for speed)
param_grid = {
    'learning_rate': [1e-4, 3e-4, 1e-3, 3e-3, 1e-2],
    'hidden_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4],
    'batch_size': [32, 64],
}

all_trials = list(itertools.product(
    param_grid['learning_rate'],
    param_grid['hidden_units'],
    param_grid['dropout_rate'],
    param_grid['batch_size'],
))

# Cap number of trials to keep runtime reasonable (set higher if you want a fuller search)
max_trials = 12
rng = np.random.default_rng(42)
if len(all_trials) > max_trials:
    trial_indices = rng.choice(len(all_trials), size=max_trials, replace=False)
    trials = [all_trials[i] for i in trial_indices]
else:
    trials = all_trials

# Use a validation SUBSET during tuning (big speedup)
val_sample_size = min(5000, len(X_val_scaled))
val_indices = rng.choice(len(X_val_scaled), size=val_sample_size, replace=False)
X_val_sample = X_val_scaled[val_indices]
y_val_sample = y_val_cat[val_indices]

hp_results = []

print("\nTesting hyperparameter configurations...")
print("=" * 70)
print(f"Grid size: {len(all_trials)} total combos; running: {len(trials)} trials (fast mode)")
print(f"Each trial: <=8 epochs w/ early stopping | val subset: {val_sample_size}")

for i, (lr, hidden_units, dropout_rate, batch_size) in enumerate(trials, start=1):
    print(f"\nTrial {i}/{len(trials)} | lr={lr:g}, hidden={hidden_units}, dropout={dropout_rate}, batch={batch_size}")

    # Prevent TF graph/memory buildup across many models
    tf.keras.backend.clear_session()

    model = create_model(
        learning_rate=float(lr),
        hidden_units=int(hidden_units),
        dropout_rate=float(dropout_rate)
    )

    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(
        X_train_sample, y_train_sample,
        validation_data=(X_val_sample, y_val_sample),
        epochs=8,
        batch_size=int(batch_size),
        callbacks=[early_stop],
        verbose=0
    )

    best_val_acc = float(max(history.history['val_accuracy']))
    best_train_acc = float(max(history.history['accuracy']))
    epochs_trained = int(len(history.history['loss']))

    hp_results.append({
        'learning_rate': float(lr),
        'hidden_units': int(hidden_units),
        'dropout_rate': float(dropout_rate),
        'batch_size': int(batch_size),
        'val_accuracy': best_val_acc,
        'train_accuracy': best_train_acc,
        'epochs': epochs_trained,
    })

    print(f"  Best val acc: {best_val_acc:.4f} | Best train acc: {best_train_acc:.4f} | Epochs: {epochs_trained}")

    # Cleanup
    del model
    gc.collect()

hp_df = pd.DataFrame(hp_results).sort_values('val_accuracy', ascending=False).reset_index(drop=True)

print("\n" + "=" * 70)
print("Hyperparameter Search Results (top 10):")
print(hp_df.head(10).to_string(index=False))

best_params = hp_df.loc[0, ['learning_rate', 'hidden_units', 'dropout_rate', 'batch_size']].to_dict()
best_val = hp_df.loc[0, 'val_accuracy']
print("\nBest hyperparameters:")
print(best_params)
print(f"Best validation accuracy: {best_val:.4f}")


In [ ]:
# Visualize hyperparameter search results
# (1) Top-N configurations (bar)
# (2) Average validation accuracy by learning rate (line)

if 'hp_df' not in globals() or hp_df.empty:
    raise ValueError("hp_df not found. Run the hyperparameter search cell first.")

top_n = min(10, len(hp_df))
top_df = hp_df.head(top_n).copy()

top_df['config'] = top_df.apply(
    lambda r: f"lr={r['learning_rate']:g}, h={int(r['hidden_units'])}, d={r['dropout_rate']}, b={int(r['batch_size'])}",
    axis=1
)

lr_summary = hp_df.groupby('learning_rate', as_index=False)['val_accuracy'].mean().sort_values('learning_rate')

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].barh(range(top_n), top_df['val_accuracy'], color='steelblue')
axes[0].set_yticks(range(top_n))
axes[0].set_yticklabels(top_df['config'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Validation Accuracy')
axes[0].set_title(f'Top {top_n} Hyperparameter Configurations')
axes[0].grid(True, alpha=0.3, axis='x')

axes[1].plot(lr_summary['learning_rate'], lr_summary['val_accuracy'], marker='o', linewidth=2)
axes[1].set_xscale('log')
axes[1].set_xlabel('Learning Rate (log scale)')
axes[1].set_ylabel('Mean Validation Accuracy')
axes[1].set_title('Mean Validation Accuracy by Learning Rate')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Part 3: Training Final Model with Best Hyperparameters


In [ ]:
# Train final model with best hyperparameters found (FAST)
# Goal: stay fast by using only 10,000 training samples + early stopping

if 'best_params' not in globals():
    raise ValueError("best_params not found. Run the hyperparameter search cell first.")

print("Training final model with best hyperparameters:")
print(best_params)
print("=" * 70)

# Ensure we have a 10,000-sample training subset (in case you skipped the search cell)
if 'X_train_sample' not in globals() or 'y_train_sample' not in globals():
    sample_size = min(10000, len(X_train_scaled))
    np.random.seed(42)
    sample_indices = np.random.choice(len(X_train_scaled), size=sample_size, replace=False)
    X_train_sample = X_train_scaled[sample_indices]
    y_train_sample = y_train_cat[sample_indices]

print(f"Using {len(X_train_sample)} training samples for final training (fast mode)")

# Create final model
final_model = create_model(
    learning_rate=float(best_params['learning_rate']),
    hidden_units=int(best_params['hidden_units']),
    dropout_rate=float(best_params['dropout_rate'])
)

# Callbacks for training
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# Train the model
history = final_model.fit(
    X_train_sample, y_train_sample,
    validation_data=(X_val_scaled, y_val_cat),
    epochs=25,
    batch_size=int(best_params['batch_size']),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\nTraining completed!")


In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy During Training')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Model Loss During Training')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
print(f"\nFinal Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


## Part 4: Model Evaluation and Performance Metrics


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = final_model.evaluate(X_test_scaled, y_test_cat, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predictions
y_pred_proba = final_model.predict(X_test_scaled, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification report
class_names = ['Below Median', 'Above Median']
print("\n" + "=" * 60)
print("Classification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


In [ ]:
# Visualize confusion matrix
class_names = ['Below Median', 'Above Median']
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Per-class accuracy
class_accuracies = cm.diagonal() / cm.sum(axis=1)
print("\nPer-Class Accuracy:")
for i, class_name in enumerate(class_names):
    print(f"  {class_name}: {class_accuracies[i]:.4f}")


## Explanation: Neural Network Application

### Task Description

We built a **feedforward neural network** for **multi-class classification** to predict song **genre** based on audio features. This is a supervised learning task where:

- **Input**: 10 audio features (danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms)
- **Output**: Music genre (e.g., pop, rock, hip-hop, classical, electronic, etc.)
- **Architecture**: Multi-layer perceptron with dropout regularization

### Network Architecture

The neural network consists of:
1. **Input Layer**: 10 neurons (one per audio feature)
2. **Hidden Layer 1**: 64 neurons with ReLU activation + Dropout (0.3)
3. **Hidden Layer 2**: 64 neurons with ReLU activation + Dropout (0.3)
4. **Hidden Layer 3**: 32 neurons with ReLU activation + Dropout (0.15)
5. **Output Layer**: N neurons with Softmax activation (one per genre class)

**Why this architecture?**
- Multiple hidden layers allow the network to learn complex non-linear relationships between audio features and genre
- Dropout layers prevent overfitting by randomly deactivating neurons during training
- ReLU activation functions enable non-linear learning while being computationally efficient
- Softmax output ensures probabilities sum to 1 for multi-class classification

**Why genre prediction is meaningful:**
- Different genres have distinct audio characteristics (e.g., classical has high acousticness, EDM has high energy)
- The model learns these patterns from the audio features
- This can be useful for music recommendation systems and playlist generation


## Explanation: Performance Metrics

### Metrics Used to Assess NN Performance

We use multiple metrics to comprehensively evaluate the neural network:

#### 1. **Accuracy**
- **Definition**: Proportion of correct predictions (TP + TN) / Total samples
- **Why it matters**: Overall model performance across all genre classes
- **Limitation**: Can be misleading with imbalanced classes

#### 2. **Loss (Categorical Cross-Entropy)**
- **Definition**: Measures the difference between predicted probabilities and true labels
- **Why it matters**: Directly optimized during training; lower is better
- **Interpretation**: Penalizes confident wrong predictions more heavily

#### 3. **Per-Class Metrics** (from Classification Report)
- **Precision**: TP / (TP + FP) - Of predicted genres, how many are correct?
- **Recall**: TP / (TP + FN) - Of actual genres, how many were found?
- **F1-Score**: Harmonic mean of precision and recall - Balanced metric
- **Support**: Number of samples in each genre

#### 4. **Confusion Matrix**
- **Definition**: Table showing actual vs predicted classifications
- **Why it matters**: Reveals which genres are confused with each other
- **Insight**: Helps identify systematic misclassification patterns (e.g., confusing rock with metal)

**Why these metrics?**
- **Accuracy** gives overall performance but can hide genre-specific issues
- **Per-class metrics** reveal if the model performs well for all genres
- **Confusion matrix** shows specific error patterns (e.g., confusing similar genres)
- Together, they provide a complete picture of model performance


## Explanation: Training Process and Hyperparameter Learning

### How the Neural Network Was Trained

#### 1. **Data Preparation**
- **Train/Validation/Test Split**: 70%/15%/15% split with stratification to maintain class distribution
- **Feature Standardization**: All features scaled to mean=0, std=1 using StandardScaler
- **One-Hot Encoding**: Labels converted to categorical format for multi-class classification

#### 2. **Hyperparameter Tuning: Learning Rate**

**Method**: Grid search over learning rates [0.0001, 0.001, 0.01, 0.1]

**Process**:
- Trained separate models with each learning rate
- Used early stopping (patience=5) to prevent overfitting
- Evaluated on validation set to find optimal value
- Selected learning rate with highest validation accuracy

**Results**: The optimal learning rate balances:
- **Too low** (0.0001): Slow convergence, may not reach optimal solution
- **Too high** (0.1): Unstable training, may overshoot optimal weights
- **Optimal** (typically 0.001-0.01): Fast convergence with stable training

#### 3. **Training Configuration**

**Optimizer**: Adam (Adaptive Moment Estimation)
- Combines benefits of momentum and adaptive learning rates
- Automatically adjusts learning rate per parameter

**Batch Size**: 32
- Processes 32 samples per gradient update
- Balances memory usage and gradient stability

**Epochs**: Up to 100 (with early stopping)
- Early stopping monitors validation loss
- Stops training if no improvement for 10 epochs
- Restores best weights to prevent overfitting

**Callbacks Used**:
- **EarlyStopping**: Prevents overfitting by stopping when validation loss stops improving
- **ReduceLROnPlateau**: Dynamically reduces learning rate if validation loss plateaus

#### 4. **Regularization Techniques**

- **Dropout**: Randomly deactivates 30% of neurons during training to prevent overfitting
- **Validation Set**: Used to monitor generalization performance during training
- **Early Stopping**: Prevents training beyond optimal point

### Hyperparameter Learning Process

1. **Initial Exploration**: Tested learning rates across orders of magnitude (0.0001 to 0.1)
2. **Validation-Based Selection**: Chose learning rate with best validation performance
3. **Final Training**: Trained final model with optimal hyperparameters on full training set
4. **Test Evaluation**: Final performance assessed on held-out test set

**Key Insight**: Learning rate is critical - too high causes instability, too low causes slow convergence. The grid search identified the sweet spot for this dataset.
